# Выбор модели для классификации вырубка/не-вырубка с использованием базовых слоев

В данном разделе обучаются и подгоняются параметры нескольких моделей для решения задачи поиска вырубок.

**Замечание:** по ходу дела тестировалось большое количество типов моделей, встроенных в 

## Обучающая выборка

Сначала посмотрим на то, насколько вообще можно обучить модель, попытавшись переобучить ее. Т.е. для начала обучим систему без разбиения выборки на обучающую и тестовую, получившийся результат даст представление о верхней границе качества.

In [ ]:
import os
import pickle

import pandas as pd
import numpy as np
import sklearn as sk

from sklearn.pipeline import Pipeline

from sklearn.metrics import confusion_matrix

from sklearn.model_selection import GridSearchCV

In [ ]:
import utilites

from utilites import (
    get_grassdata_path,
    get_location_name,
    get_ll_location_name,
    get_location_path,
    get_gisbase_path
)

import grasslib
reload(grasslib)

from grasslib import GRASS

In [ ]:
grs = GRASS(gisbase=get_gisbase_path(), 
            dbase=get_grassdata_path(), 
            location=get_location_name(),
            mapset='basemaps'
)

## Обучающая выборка

Создадим обучающую выборку, которая будет классифицировать данные по рубкам за зиму 15-16 годов.

In [ ]:
print grs.grass.read_command('g.region', region='all_scenes@landsat', flags='p')

Слой зимних рубок, которые должны быть обнаружены зимой 15-16 годов:

In [ ]:
grs.grass.run_command('v.to.rast', 
                      input='train', output='tr1516.bin', 
                      where="winter='w1516'", use='val', value=1, 
                      overwrite=True)
grs.grass.run_command('r.null', map='tr1516.bin', null=0)
print grs.grass.read_command('r.report', map='tr1516.bin', units='c')

Слой алармов, которые должны быть обнаружены зимой 15-16гг:

In [ ]:
# В начале весны ловятся прошлогодние зимние рубки, поэтому выкинем их из обучающего множества
grs.grass.run_command('r.mapcalc', 
                      expression="tr1516.alrm = (alarm_2015@umd_alarm > 170)", 
                      overwrite=True)
print grs.grass.read_command('r.report', map='tr1516.alrm', units='c')

In [ ]:
inputs = grs.grass.list_strings('rast', pattern="diff1615*")
inputs

In [ ]:
print grs.grass.read_command('g.region', region='all_scenes@landsat', res=90, flags='p')

In [ ]:
X_train = grs.rasters_to_array(inputs)

In [ ]:
y_train = grs.rasters_to_array(['tr1516.alrm']).ravel()

## Случайный лес

In [ ]:
from sklearn.ensemble import RandomForestClassifier

In [ ]:
forest = RandomForestClassifier(n_estimators=100, random_state=1, n_jobs=7)

In [ ]:
forest.fit(X_train, y_train)

In [ ]:
y_pred = forest.predict(X_train)
print confusion_matrix(y_train, y_pred)
grs.grass.run_command('g.remove', type='rast', name='forest.result90m', flags='f')
grs.array_to_rast(arr=y_pred, map_name='forest.result90m')

Сохраним модель в файл. Применим модель к данным (на которых она обучалась) и посчитаем матрицу ошибок.

In [ ]:
pickle.dump(forest, open(os.path.join('Models', 'forest_classifier.pkl'), 'wb'), protocol=2)
model = pickle.load(open(os.path.join('Models', 'forest_classifier.pkl'), 'rb'))
tmp = model.predict(X_train)

confusion_matrix(y_train, tmp)
del(tmp)

То, что модель показала низкий процент ошибок - хороший знак, как минимум рубки видим, что отличаются от остального. Правда, это та верхняя граница точности, которую можно надеятся получить при помощи данной модели: скорее всего произошло переобучение и предсказательная способность модели низка.

## Растры прогнозов

Применим модель и сохраним растры прогнозов.

In [ ]:
print grs.grass.read_command('g.region', region='all_scenes@landsat', res=60, flags='p')

In [ ]:
model = pickle.load(open(os.path.join('Models', 'forest_classifier.pkl'), 'rb'))

In [ ]:
X_train = grs.rasters_to_array(inputs)

Как показала практика, расчет результатов на нормальном разрешении не лезет в память, поэтому разобьем множест во на куски, потом склеим результаты по каждому куску в одно целое.

In [ ]:
p1, p2, p3 = X_train.shape[0]/4, X_train.shape[0]/2, X_train.shape[0]*3/4
print (p1, p2, p3)

X1 = X_train[: p1, :]
X2 = X_train[p1:p2, :]
X3 = X_train[p2:p3, :]
X4 = X_train[p3:, :]

In [ ]:
result1 = model.predict(X1)

In [ ]:
result2 = model.predict(X2)

In [ ]:
result3 = model.predict(X3)

In [ ]:
result4 = model.predict(X4)

In [ ]:
y_pred = np.concatenate((result1, result2, result3, result4))

In [ ]:
del(result1, result2, result3, result4)

In [ ]:
grs.grass.run_command('g.remove', type='rast', name='forest.result60m', flags='f')
grs.array_to_rast(arr=y_pred, map_name='forest.result60m')

Разрешение в 30 метров:

In [ ]:
print grs.grass.read_command('g.region', region='all_scenes@landsat', res=30, flags='p')

In [ ]:
# del(X_train)
X_train = grs.rasters_to_array(inputs)

In [ ]:
size = X_train.shape[0]
bins = np.linspace(0, size).astype(np.int)
bins

In [ ]:
i = 0
result = model.predict(X_train[bins[i]: bins[i+1], :])

In [ ]:
last = len(bins) - 2
for i in range(1, last):
    r = model.predict(X_train[bins[i]: bins[i+1], :])
    result = np.concatenate((result, r))

r = model.predict(X_train[bins[last]:, :])
result = np.concatenate((result, r))

In [ ]:
len(result)

In [ ]:
grs.grass.run_command('g.remove', type='rast', name='forest.result30m', flags='f')
grs.array_to_rast(arr=result, map_name='forest.result30m')

## Обучение с перекрестной проверкой

Обучим модель "по-правильному" с использованием перекрестной проверки и тестового множества.

In [ ]:
print grs.grass.read_command('g.region', region='all_scenes@landsat', flags='p')

Разобьем данные на обучающее и тестовое множества. Чтобы избежать проблем с пространственной автокорреляцией, когда пикселы, попавшие в одну и ту же рубку будут похожи по своим свойствам в силу пространственно-временной близости события, сделаем так, что в обучающее и тестовое множества будут попадать полигоны, а не пиксели. Для этого каждому полигону рубки назначим уникальный идентификатор и будем разбрасывать пиксели по обучающему и тестовму множествам по этим идентификаторам.

In [ ]:
grs.grass.run_command('r.null', setnull=0, map='tr1516.alrm')

grs.grass.run_command('r.to.vect', input='tr1516.alrm', output='tr1516_alrm', type='area', 
                      overwrite=True)
grs.grass.run_command('v.to.rast', input='tr1516_alrm', output='tr1516.alrm.uniq', use='cat', 
                      overwrite=True)

grs.grass.run_command('r.null', null=0, map='tr1516.alrm')
grs.grass.run_command('r.null', null=0, map='tr1516.alrm.uniq')

In [ ]:
print grs.grass.read_command('r.report', map='tr1516.alrm.uniq', units='c')

Уменьшим рабочее разрешение, иначе слишком долго обучать модели:

In [ ]:
print grs.grass.read_command('g.region', region='all_scenes@landsat', res=90, flags='p')

In [ ]:
data = grs.rasters_to_array(inputs + ['tr1516.alrm.uniq'])

Пикселям, попавшим в категории "не рубки" тоже назначим уникальные идентификаторы:

In [ ]:
mask = (data[:, -1] == 0)

In [ ]:
count0 = np.count_nonzero(mask)
nums = -np.random.rand(count0)
data[mask, -1] = nums

In [ ]:
cats = np.unique(data[:, -1])

Собственно разбиение на множества:

In [ ]:
train_cat, test_cat = sk.model_selection.train_test_split(cats, test_size=0.33, random_state=1)

In [ ]:
len (train_cat), len(test_cat), len(cats)

In [ ]:
test_data = data[np.in1d(data[:, -1], test_cat), :]
train_data = data[np.in1d(data[:, -1], train_cat), :]

test_data.shape, train_data.shape

In [ ]:
X_test = test_data[:, :-1]
y_test = test_data[:, -1].ravel()

X_test.shape

In [ ]:
X_train = train_data[:, :-1]
y_train = train_data[:, -1].ravel()

X_train.shape

In [ ]:
mask = y_train > 0
y_train[mask] = 1

mask = y_train < 0
y_train[mask] = 0

In [ ]:
mask = y_test > 0
y_test[mask] = 1

mask = y_test < 0
y_test[mask] = 0

Обучим одну модель (случайный лес, параметры по умолчанию), оценим ошибку и ее разброс при помощи перекрестной проверки:

In [ ]:
pipe_forest = Pipeline([
        # ('pca', PCA(n_components=10)), 
        ('clf', RandomForestClassifier(random_state=1, n_jobs=7))
])


scores = sk.model_selection.cross_val_score(
    estimator=pipe_forest,
    X=X_train,
    y=y_train,
    cv=10
)

print('CV accuracy scores: %s' % scores)
print('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores) * 2))

Проверим качество на теством множестве:

In [ ]:
pipe_forest.fit(X_train, y_train)
y_pred = pipe_forest.predict(X_test)
confusion_matrix(y_test, y_pred)

При помощи перекрестной проверки найдем модель с оптимальным числом деревьев в лесу (**осторожно, так просто из любопытства не запускать: обучается 4 дня!**):

In [ ]:
param_range = range(2, 21) + [25, 30, 40, 50, 60, 70, 80, 90, 100]
param_grid = [{'clf__n_estimators': param_range}]
gs = GridSearchCV(
    estimator=pipe_forest,
    param_grid=param_grid,
    scoring='accuracy',
    cv=10
)
gs = gs.fit(X_train, y_train)
print(gs.best_score_)
print(gs.best_params_)

### Прогноз

Проверим результат на тестовом множестве:

In [ ]:
best_model = gs.best_estimator_
y_pred = best_model.predict(X_test)
confusion_matrix(y_test, y_pred)

Как и ранее: если все пометить, как "не рубки", то ошибка будет низкой, но реальное качество поиска рубок на нуле.

Запустим работу модели на всех данных, сохраним резульаты в растры:

In [ ]:
print grs.grass.read_command('g.region', region='all_scenes@landsat', res=30, flags='p')

In [ ]:
X_train = grs.rasters_to_array(inputs)
size = X_train.shape[0]
bins = np.linspace(0, size).astype(np.int)
print bins

i = 0
result = best_model.predict(X_train[bins[i]: bins[i+1], :])

last = len(bins) - 2
for i in range(1, last):
    r = best_model.predict(X_train[bins[i]: bins[i+1], :])
    result = np.concatenate((result, r))

r = best_model.predict(X_train[bins[last]:, :])
result = np.concatenate((result, r))
print len(result)


# grs.grass.run_command('g.remove', type='rast', name='forest.result30m', flags='f')
grs.array_to_rast(arr=result, map_name='forest.cv.result30m')